In [50]:
import MetaTrader5 as mt5
import pandas as pd
from peaks import calculate_peak,print_horario
import numpy as np
import talib as ta
from scipy.signal import argrelextrema
from scipy.signal import find_peaks
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

if not mt5.initialize(path = r'C:\Program Files\Zero Markets MetaTrader 5 Terminal\terminal64.exe'):
    print("initialize() failed")
    mt5.shutdown()

In [2]:
ativo = 'EURUSD'
df = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_M1, 0, 1000))
df['Data'] = pd.to_datetime(df['time'], unit='s').apply(lambda x: str(x))
df = df[['Data','open','high','low','close','real_volume']]
df.columns = ['Data','Open','High','Low','Close','Vol']
df = df.rename(columns = {'real_volume':'vol'})
df['RSI'] = ta.RSI(df['Close'],timeperiod = 14)

In [3]:
stats = calculate_peak(df.iloc[:],dist=2,plot=True,tam_stop = 0,valley = False)
print_horario(stats)

Dist: 300-- Tam_stop: 0.0003 -- pontos_total: -0.0005199999999985216 -- retorno: -0.02985726243980229
    Hora_h  qtd_operacoes  taxa_acerto
0        8              2     0.500000
1        9              1     1.000000
2       10              1     0.000000
3       11              4     0.500000
4       12              1     0.000000
5       13              7     0.571429
6       14             11     0.636364
7       15              5     0.400000
8       16              4     0.500000
9       17              4     0.750000
10      18              2     0.500000
11      19              3     0.666667
12      20              9     0.444444
13      22              3     0.666667
14      23              4     0.500000


In [15]:
df.iloc[:15]['Close']

0     1.08670
1     1.08672
2     1.08666
3     1.08666
4     1.08662
5     1.08658
6     1.08662
7     1.08661
8     1.08665
9     1.08665
10    1.08674
11    1.08674
12    1.08677
13    1.08693
14    1.08687
Name: Close, dtype: float64

In [42]:
def teste(x):
    t = 0
    for a in range(4):
        if x.iloc[a+1] > x.iloc[a]:
            t+=1
    return t

df.iloc[:30][["Close"]].rolling(5).apply(teste)

,Close
0,NaN
1,NaN
2,NaN
3,NaN
4,1.0
5,0.0
6,1.0
7,1.0
8,2.0
9,2.0


In [43]:
df.iloc[:30][["Close"]].rolling(-5).apply(teste)

ValueError: window must be an integer 0 or greater

In [60]:
time_series = df['Close']
indices = find_peaks(time_series,distance = 10)[0]

fig = go.Figure()
fig.add_trace(go.Scatter(
    y=time_series,
    mode='lines',
    name='Original Plot'
))

fig.add_trace(go.Scatter(
    x=indices,
    y=[time_series[j] for j in indices],
    mode='markers',
    marker=dict(
        size=8,
        color='red',
        symbol='cross'
    ),
    name='Detected Peaks'
))

fig.show()